# Week 02 Assignment glucose level data

Welcome to week two of this course programming 1. You will learn about time related data wrangling with pandas and you will learn to visualize with bokeh. This week will be focussed around missing data. Concretely, you will preprocess the glucose json file using interpolation to impute in order to conduct visual analysis. Learning outcomes:

- load a json dataset 
- typecast the Pandas DataFrame to appropiate data types
- inspect the dataset for quality and metadata information
- add a column with interpolated data in Pandas DataFrame
- perform visual analysis

The assignment consists of 6 parts:

- [part 1: load the data](#0)
     - [Exercise 1.1](#ex-11)
- [part 2: prepare for inspection](#1)
     - [Exercise 2.1](#ex-21)
- [part 3: inspect the data](#2)
     - [Exercise 3.1](#ex-31)
- [part 4: interpolate the data](#3)
     - [Exercise 4.1](#ex-41)
- [part 5: visualize the data](#4)
     - [Exercise 5.1](#ex-51)
- [part 6: Challenge](#5)
     - [Exercise 6.1](#ex-61)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
To pass the assingnment you need to a score of 60%. 


<a name='0'></a>
## Part 1: Load the data

Instructions: Load the json datafile `glucose.json` into a pandas dataframe. Check your dataframe with a `.head()` to compare with the expected outcome

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>json.load() method reads a file, pd.read_json converts it to a Pandas DataFrame</li>
    <li>when loading into a Pandas DataFrame use records orientation </li>
</ul>
</details>

<a name='ex-11'></a>
### Code your solution

In [913]:
import pandas as pd
import numpy as np 
import json as json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import gridplot, row
from bokeh.models import CheckboxGroup, CustomJS, Panel, Tabs, Label
from bokeh.io import output_notebook
output_notebook()


         
with open("../data/glucose.json", "r") as filename:
    json_file = json.load(filename)
    df = pd.read_json(json_file)
    
df.head()



Loading BokehJS ...

,ID,time,recordtype,glucose
0,2845.0,2019-04-25 00:08,1,109
1,2850.0,2019-04-25 00:50,1,
2,2877.0,2019-04-25 07:02,1,123
3,2881.0,2019-04-25 07:34,1,158
4,2886.0,2019-04-25 08:19,1,


#### Expected outcome: 

<a name='1'></a>
## Part 2: Prepare the data

Check the datatypes of your dataframe. The `glucose` field should be an integer, the `time` field should have a datetime format. If the datatypes are different you should typecast them to the right format.
Make sure that your dataset is sorted by the time column


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use astype() method or pandas.DataFrame.to_datetime() for instance</li>
    <li>make sure that the empty spaces are filled with NaN. Use errors='coerce'</li>
    <li>set_index(), sort_index() and reset_index() are helpful to sort on index</li>
</ul>
</details>

<a name='ex-21'></a>
### Code your solution

In [914]:
# CODE YOUR SOLUTION HERE
#print(df)

# df.info()

# glucose should be interger

#df["ID"] = pd.to_numeric(df["ID"], errors= "coerce")
df = df.astype({'ID': 'int64'})
#df = df.astype({'glucose': 'int64'})
df = df.astype({'time': 'datetime64[ns]'})
df["glucose"] = pd.to_numeric(df["glucose"], errors= "coerce")
# df["time"] = pd.to_datetime(df["time"], errors= "coerce")
df.sort_values("time")
df.info()
#print(df)

# I have no idea what happend to my type 0 ID's
# Recordtype 0 has weird IDs and all glucose values are missing / recordtype 1 has normal IDs, and glucose values, but still misses a couple of values. 



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ID          136 non-null    int64         
 1   time        136 non-null    datetime64[ns]
 2   recordtype  136 non-null    int64         
 3   glucose     52 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 4.4 KB


#### Expected outcome: 

<a name='2'></a>
## Part 3: Inspect the data

Now that we prepared the data we are going to inspect the data to get more familiar with the data. You are required to do the following

- inspect the percentage missing data for glucose
- what is the relationship between recordtype and glucose value?
- what is the relationship between ID and glucose value?

Code the solutions to your answers. Create meaningful overviews or statistics

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>In the week 01 assignment some functions were explained to inspect missing values</li>
    <li>In the week 01 assignment some functions were explained to groupby value</li>
</ul>
</details>

<a name='ex-31'></a>
### Code your solution

In [915]:
#CODE YOUR SOLUTION HERE

#print(df)

# percebtage missing from the df
percent_missing = df.isnull().sum() * 100 / len(df)
# missing_value_df = pd.DataFrame({'column_name': df.columns,
#                                  'percent_missing': percent_missing})

print("table with missing values ", percent_missing)

# relation between glucose and record type
data_one = len(pd.DataFrame(df.groupby('recordtype')))
print("Number of record types :", data_one)

selected_columns_record =df[["recordtype", "glucose"]]
#print(selected_columns_record)


# relation between glucose and ID
data_two = len(pd.DataFrame(df.groupby('ID')))
print("Number of ID's :", data_two)

selected_columns_ID =df[["ID", "glucose"]]
#print(selected_columns_ID)
#selected_columns_ID.head()





table with missing values  ID             0.000000
time           0.000000
recordtype     0.000000
glucose       61.764706
dtype: float64
Number of record types : 2
Number of ID's : 55


In [916]:
# # Delete missing values form the dataset
# df = df.dropna()

# Check result
# #print(df)
# df.head()

#### Expected outcome percentage missing data
0.6176470588235294

<a name='3'></a>
## Part 4: Interpolate the data

A lot of data is missing. Use interpolation to fill the missing values. Create a new column with the interpolated data. Take an argumentative approach. Select an interpolation method that suits the nature of the data and explain your choice. Mind you that the expected outcome of the interpolation values can differ from the example below

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use Pandas.DataFrame.interpolate() method</li>
</ul>
</details>

<a name='ex-41'></a>
### Code your solution

In [917]:
#Save interpolate to a variable
interpolate = df.glucose.interpolate(method="linear")
#print(interpolate)
 

# Add the list to the datframe
df["interpolation"] = interpolate

# Check result
print(df)




                      ID                time  recordtype  glucose  \
0                   2845 2019-04-25 00:08:00           1    109.0   
1                   2850 2019-04-25 00:50:00           1      NaN   
2                   2877 2019-04-25 07:02:00           1    123.0   
3                   2881 2019-04-25 07:34:00           1    158.0   
4                   2886 2019-04-25 08:19:00           1      NaN   
..                   ...                 ...         ...      ...   
131 -9223372036854775808 2019-04-25 22:47:00           0      NaN   
132 -9223372036854775808 2019-04-25 23:02:00           0      NaN   
133 -9223372036854775808 2019-04-25 23:18:00           0      NaN   
134 -9223372036854775808 2019-04-25 23:33:00           0      NaN   
135 -9223372036854775808 2019-04-25 23:48:00           0      NaN   

     interpolation  
0            109.0  
1            116.0  
2            123.0  
3            158.0  
4            148.5  
..             ...  
131          111.0  
132

#### Example outcome

<a name='4'></a>
## Part 5: Plot the data

Create a plot with the original data and the interpolated data. Consider what the best representation is for visualisation of actual values and modelled/imputed values. An example of such a plot is given below. This plot however is not considered the best practice. 

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>figure(x_axis_type='datetime') automatically makes nices labels of the datetime data</li>
</ul>
</details>

<a name='ex-51'></a>
### Code your solution

In [918]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.plotting import ColumnDataSource
from datetime import datetime as dt
output_notebook()


Loading BokehJS ...

In [919]:
df.info()
df.head
#print(df.interpolated)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             136 non-null    int64         
 1   time           136 non-null    datetime64[ns]
 2   recordtype     136 non-null    int64         
 3   glucose        52 non-null     float64       
 4   interpolation  136 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 5.4 KB


<bound method NDFrame.head of                       ID                time  recordtype  glucose  \
0                   2845 2019-04-25 00:08:00           1    109.0   
1                   2850 2019-04-25 00:50:00           1      NaN   
2                   2877 2019-04-25 07:02:00           1    123.0   
3                   2881 2019-04-25 07:34:00           1    158.0   
4                   2886 2019-04-25 08:19:00           1      NaN   
..                   ...                 ...         ...      ...   
131 -9223372036854775808 2019-04-25 22:47:00           0      NaN   
132 -9223372036854775808 2019-04-25 23:02:00           0      NaN   
133 -9223372036854775808 2019-04-25 23:18:00           0      NaN   
134 -9223372036854775808 2019-04-25 23:33:00           0      NaN   
135 -9223372036854775808 2019-04-25 23:48:00           0      NaN   

     interpolation  
0            109.0  
1            116.0  
2            123.0  
3            158.0  
4            148.5  
..             

In [922]:
# Splitting time and date column for easier plotting 
#df['new_date'],df['new_time'] = zip(*[(d.date(), d.time()) for d in df['time']])

# Renovimg values to check for better distribution
#df.drop(df.index[df['interpolation'] == 111], inplace = True)

#normal glucopse
x = df.time
y = df.glucose
y2 = df.interpolation

p1 = figure(title="Glucose distribution over time", x_axis_label='Time', y_axis_label='Glucose(mmol/l)', x_axis_type='datetime')
p1.line(x, y, legend_label="Glucose.", line_width=2)
mytext = Label(x=70, y=70, text="Timeseries displaying glucose measurments over a 24h period")
p1.add_layout(mytext)
# show(p1)

# # interpolated glucose
p2 = figure(title="interpolated glucose distribution over time", x_axis_label='Time', y_axis_label='Glucose(mmol/l)', x_axis_type='datetime')
p2.line(x, y2, legend_label="Interpolated glucose.", line_width=2)
mytext = Label(x=70, y=70, text="Timeseries displaying linear interpolated glucose measurments over a 24h period")
p2.add_layout(mytext)
# show(p2)


# Plot panel of glucose and interpolated
show(row(p1, p2))





<a name='6'></a>
## Part 6: Challenge

It might even be interesting to introduce a widget in which you can select different methods to interpolate.
1. Can you improve the interpolation by choosing an other method?
2. Can you add an rolling mean line? 
2. Can you improve the plot by making it interactive?

<a name='ex-61'></a>
### Code your solution

In [921]:
# Trying diffrent methods of interpolation and assigning to the dataframe

a_int = df.glucose.interpolate(method="polynomial", order = 2)
df["poly_int"] = a_int

b_int = df.glucose.interpolate(method="pad", order = 2)
df["padded_int"] = b_int

c_int = df.glucose.interpolate(method="quadratic", order = 2)
df["quad_int"] = c_int

d_int = df.glucose.interpolate(method="spline", order = 2)
df["spline_int"] = d_int

# Check result
#print(df)

# assigning the diffrent interpolations to a variable
x = df.time
y = df.glucose
y2 = df.interpolation
y3 = df.padded_int
y4 = df.poly_int
y5 = df.quad_int
y6 = df.spline_int

#Creating figures of the the diffrent interpolated variables over time

p1 = figure(title="Glucose distribution over time", x_axis_label='Time(H)', y_axis_label='Glucose(mmol/l)', x_axis_type='datetime')
p1.line(x, y, legend_label="Glucose.", line_width=2)
p1.line()
tab1 = Panel(child=p1, title="Glucose")
# show(p1)

# # interpolated glucose
p2 = figure(title="Linear interpolated glucose distribution over time", x_axis_label='Time(H)', y_axis_label='Glucose(mmol/l)', x_axis_type='datetime')
p2.line(x, y2, legend_label="Interpolated glucose.", line_width=2)
tab2 = Panel(child=p2, title="Linear interpolation")
# show(p2)

p3 = figure(title="Padded interpolated glucose distribution over Time", x_axis_label='Time(H)', y_axis_label='Glucose(mmol/l)', x_axis_type='datetime')
p3.line(x, y3, legend_label="Interpolated glucose.", line_width=2)
tab3 = Panel(child=p3, title="padded interpolation")
# show(p3)

p4 = figure(title="polynomial interpolated glucose distribution over time", x_axis_label='Time(H)', y_axis_label='Glucose(mmol/l)', x_axis_type='datetime')
p4.line(x, y4, legend_label="Interpolated glucose.", line_width=2)
tab4 = Panel(child=p4, title="polynomial interpolation")
# show(p4)

p5 = figure(title="quadratic interpolated glucose distribution over time", x_axis_label='Time(H)', y_axis_label='Glucose(mmol/l)', x_axis_type='datetime')
p5.line(x, y5, legend_label="Interpolated glucose.", line_width=2)
tab5 = Panel(child=p5, title="quadratic interpolation")
# show(p5)

p6 = figure(title="spline interpolated glucose distribution over time", x_axis_label='Time(H)', y_axis_label='Glucose(mmol/l)', x_axis_type='datetime')
p6.line(x, y6, legend_label="Interpolated glucose.", line_width=2)
tab6 = Panel(child=p6, title="spline interpolation")

# Showing plot with tabs selecting each possible interpolation.
 
show(Tabs(tabs=[tab1, tab2, tab3, tab4, tab5, tab6]))

print("Graphs show no major diffrence between interpolation mathods, but quadratic interpolating shows no weird line, so is the better method for visualization")

# adding a rolling meam to the linear interpolated glucose data

#adding a rolling mean
df[ 'rolling_avg' ] = df.quad_int.rolling( 7).mean()

#setting data to variable
x = df.time
y = df.quad_int
y2 = df.rolling_avg

# plotting interpolated glucose with rolling mean
p1 = figure(title="Glucose distribution over time", x_axis_label='Time', y_axis_label='Glucose(mmol/l)', x_axis_type='datetime')
p1.line(x, y, legend_label="Glucose.", line_width=2)
p1.line(x, y2, legend_label="rolling average.", line_width=2, color = "red")
show(p1)


Graphs show no major diffrence between interpolation mathods, but quadratic interpolating shows no weird line, so is the better method for visualization
